In [2]:
import time
import json
import redis
import asyncio
import numpy as np
from datetime import datetime
from aredis import StrictRedis

In [3]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
ar = StrictRedis(host='127.0.0.1', port=6379, db=8)

In [3]:

for key in rd.keys():
    rd.delete(key)

In [4]:
for key in rd.keys():
    print(key)

b'20210625_is_closed_day'
b'hq_20210625_symbols_length'
b'hq_20210625_check_points_length'
b'hq_assist_count'


In [3]:
await asyncio.sleep(int(time.mktime(time.strptime(f'{time.strftime("%Y-%m-%d")} 09:14:00', '%Y-%m-%d %H:%M:%S'))) - time.time()) 

date = time.strftime('%Y%m%d')

check_points_length = int(rd.get(f'hq_{date}_check_points_length'))

assist_count = int(rd.get('hq_assist_count'))

msg = {
    'command': 'snapshotting',
    'date': time.strftime('%Y%m%d')
}

for _ in range(assist_count):
    rd.lpush(f'hq_assist_{_}', json.dumps(msg))

In [6]:
await ar.pubsub_channels()

[b'hq_assist_4_snapshotting',
 b'hq_assist_5_snapshotting',
 b'hq_assist_3_snapshotting',
 b'hq_assist_0_snapshotting',
 b'hq_assist_6_snapshotting',
 b'hq_assist_2_snapshotting',
 b'hq_assist_1_snapshotting']

In [ ]:

def message_handler(data):

    assist_idx = str(message['channel']).split('_')[2]
    data = json.loads(message['data'])

    if data['status'] == 'successful':
        check_point_idx = int(data['idx'])
        check_point = data['check_point']
        status[check_point_idx] += 1

        if status[check_point_idx] == assist_count:
            pass

        print('handler:', assist_idx, status[check_point_idx], message['data'])

In [7]:
async def snapshot_handler(assist_count, check_points_length, handler):
    status = np.zeros(check_points_length, dtype=int)
    status

    def pre_handler(message):
        assist_idx = str(message['channel']).split('_')[2]
        data = json.loads(message['data'])
        check_point_idx = int(data['idx'])
        
        if data['status'] == 'successful':
            status[check_point_idx] += 1
        
            if status[check_point_idx] == assist_count:
                handler(data)
            print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")} ss_handler:', assist_idx, check_point_idx, status[check_point_idx])  
        else:
            print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")} ss_handler:', assist_idx, check_point_idx, message['data'])
            
    p = ar.pubsub()
    await p.psubscribe(**{'hq_assist_*_snapshotting': pre_handler})

    while status[-1] != assist_count:

        message = await p.get_message()

#         if np.sum(status) >= assist_count:
#             break

    await asyncio.sleep(5)
    await p.punsubscribe('hq_assist_*_snapshotting')
    p.close()

In [ ]:
await snapshot_handler(assist_count, check_points_length)

In [ ]:
asyncio.create_task(snapshot_handler(assist_count, check_points_length))

In [ ]:
date = time.strftime('%Y%m%d')
date

In [ ]:
check_points_length = int(rd.get(f'hq_{date}_check_points_length'))
check_points_length

In [ ]:
assist_count = int(rd.get('hq_assist_count'))
assist_count

In [ ]:
msg = {
    'command': 'quit'
}


In [ ]:
msg = {
    'command': 'snapshotting',
    'date': time.strftime('%Y%m%d')
}
msg

In [ ]:
for _ in range(assist_count):
    rd.lpush(f'hq_assist_{_}', json.dumps(msg))

In [4]:
async def incremental_save(assist_count, check_points_length):
    subs = [ ar.pubsub() for assist_idx in range(assist_count) ]
    
    results = await asyncio.gather(*[ sub.subscribe(f'hq_assist_{_}_snapshotting') for _,sub in enumerate(subs) ])
    results = await asyncio.gather(*[ sub.parse_response() for sub in subs ])
    assert all([ result[0] == b'subscribe' and result[-1]==1 for result in results])
    
    date = time.strftime('%Y%m%d')
    while True:
        results = await asyncio.gather(*[ sub.parse_response() for sub in subs ])
        idx = json.loads(results[0][2])['idx']
        msg = {
            'command': 'incremental_save',
            'date': time.strftime('%Y%m%d'),
            'idx': idx
        }
        rd.lpush(f'hq_assist_0', json.dumps(msg))
        
        if idx+1 >= check_points_length:
            break
    
    results = await asyncio.gather(*[ sub.unsubscribe(f'hq_assist_{_}_snapshotting') for _,sub in enumerate(subs) ])
    results = await asyncio.gather(*[ sub.parse_response() for sub in subs ])
    assert all([ result[0] == b'unsubscribe' and result[-1]==0 for result in results])

In [5]:
asyncio.create_task(incremental_save(assist_count, check_points_length))

<Task pending name='Task-2' coro=<incremental_save() running at <ipython-input-4-4d35b30c1663>:1>>

In [11]:
await ar.exists('foo')

False

handler: 6 1 b'{"status": "successful", "idx": 558, "check_point": 1624500375}'
handler: 0 2 b'{"status": "successful", "idx": 558, "check_point": 1624500375}'
handler: 3 3 b'{"status": "successful", "idx": 558, "check_point": 1624500375}'
handler: 1 4 b'{"status": "successful", "idx": 558, "check_point": 1624500375}'
handler: 2 5 b'{"status": "successful", "idx": 558, "check_point": 1624500375}'
handler: 5 6 b'{"status": "successful", "idx": 558, "check_point": 1624500375}'
handler: 4 7 b'{"status": "successful", "idx": 558, "check_point": 1624500375}'


In [14]:
while await ar.get('lkdf') is None:
    await asyncio.sleep(1)

handler: 0 1 b'{"status": "successful", "idx": 864, "check_point": 1624501905}'
handler: 4 2 b'{"status": "successful", "idx": 864, "check_point": 1624501905}'
handler: 3 3 b'{"status": "successful", "idx": 864, "check_point": 1624501905}'
handler: 1 4 b'{"status": "successful", "idx": 864, "check_point": 1624501905}'
handler: 2 5 b'{"status": "successful", "idx": 864, "check_point": 1624501905}'
handler: 6 6 b'{"status": "successful", "idx": 864, "check_point": 1624501905}'
handler: 5 7 b'{"status": "successful", "idx": 864, "check_point": 1624501905}'


True

handler: 4 1 b'{"status": "successful", "idx": 865, "check_point": 1624501910}'
handler: 1 2 b'{"status": "successful", "idx": 865, "check_point": 1624501910}'
handler: 2 3 b'{"status": "successful", "idx": 865, "check_point": 1624501910}'
handler: 3 4 b'{"status": "successful", "idx": 865, "check_point": 1624501910}'
handler: 0 5 b'{"status": "successful", "idx": 865, "check_point": 1624501910}'
handler: 6 6 b'{"status": "successful", "idx": 865, "check_point": 1624501910}'
handler: 5 7 b'{"status": "successful", "idx": 865, "check_point": 1624501910}'
handler: 3 1 b'{"status": "successful", "idx": 866, "check_point": 1624501915}'
handler: 6 2 b'{"status": "successful", "idx": 866, "check_point": 1624501915}'
handler: 2 3 b'{"status": "successful", "idx": 866, "check_point": 1624501915}'
handler: 4 4 b'{"status": "successful", "idx": 866, "check_point": 1624501915}'
handler: 0 5 b'{"status": "successful", "idx": 866, "check_point": 1624501915}'
handler: 1 6 b'{"status": "successful", 

handler: 4 1 b'{"status": "successful", "idx": 880, "check_point": 1624501985}'
handler: 2 2 b'{"status": "successful", "idx": 880, "check_point": 1624501985}'
handler: 6 3 b'{"status": "successful", "idx": 880, "check_point": 1624501985}'
handler: 0 4 b'{"status": "successful", "idx": 880, "check_point": 1624501985}'
handler: 5 5 b'{"status": "successful", "idx": 880, "check_point": 1624501985}'
handler: 1 6 b'{"status": "successful", "idx": 880, "check_point": 1624501985}'
handler: 3 7 b'{"status": "successful", "idx": 880, "check_point": 1624501985}'
handler: 4 1 b'{"status": "successful", "idx": 881, "check_point": 1624501990}'
handler: 0 2 b'{"status": "successful", "idx": 881, "check_point": 1624501990}'
handler: 3 3 b'{"status": "successful", "idx": 881, "check_point": 1624501990}'
handler: 6 4 b'{"status": "successful", "idx": 881, "check_point": 1624501990}'
handler: 1 5 b'{"status": "successful", "idx": 881, "check_point": 1624501990}'
handler: 2 6 b'{"status": "successful", 

handler: 1 7 b'{"status": "successful", "idx": 894, "check_point": 1624502055}'
handler: 0 1 b'{"status": "successful", "idx": 895, "check_point": 1624502060}'
handler: 6 2 b'{"status": "successful", "idx": 895, "check_point": 1624502060}'
handler: 4 3 b'{"status": "successful", "idx": 895, "check_point": 1624502060}'
handler: 3 4 b'{"status": "successful", "idx": 895, "check_point": 1624502060}'
handler: 5 5 b'{"status": "successful", "idx": 895, "check_point": 1624502060}'
handler: 2 6 b'{"status": "successful", "idx": 895, "check_point": 1624502060}'
handler: 1 7 b'{"status": "successful", "idx": 895, "check_point": 1624502060}'
handler: 4 1 b'{"status": "successful", "idx": 896, "check_point": 1624502065}'
handler: 6 2 b'{"status": "successful", "idx": 896, "check_point": 1624502065}'
handler: 5 3 b'{"status": "successful", "idx": 896, "check_point": 1624502065}'
handler: 3 4 b'{"status": "successful", "idx": 896, "check_point": 1624502065}'
handler: 0 5 b'{"status": "successful", 

...

handler: 4 1 b'{"status": "successful", "idx": 2934, "check_point": 1624517645}'
handler: 3 2 b'{"status": "successful", "idx": 2934, "check_point": 1624517645}'
handler: 1 3 b'{"status": "successful", "idx": 2934, "check_point": 1624517645}'
handler: 2 4 b'{"status": "successful", "idx": 2934, "check_point": 1624517645}'
handler: 5 5 b'{"status": "successful", "idx": 2934, "check_point": 1624517645}'
handler: 0 6 b'{"status": "successful", "idx": 2934, "check_point": 1624517645}'
handler: 6 7 b'{"status": "successful", "idx": 2934, "check_point": 1624517645}'
handler: 4 1 b'{"status": "successful", "idx": 2935, "check_point": 1624517650}'
handler: 5 2 b'{"status": "successful", "idx": 2935, "check_point": 1624517650}'
handler: 3 3 b'{"status": "successful", "idx": 2935, "check_point": 1624517650}'
handler: 2 4 b'{"status": "successful", "idx": 2935, "check_point": 1624517650}'
handler: 6 5 b'{"status": "successful", "idx": 2935, "check_point": 1624517650}'
handler: 1 6 b'{"status": "s

handler: 1 6 b'{"status": "successful", "idx": 2948, "check_point": 1624517715}'
handler: 3 7 b'{"status": "successful", "idx": 2948, "check_point": 1624517715}'
handler: 6 1 b'{"status": "successful", "idx": 2949, "check_point": 1624517720}'
handler: 0 2 b'{"status": "successful", "idx": 2949, "check_point": 1624517720}'
handler: 2 3 b'{"status": "successful", "idx": 2949, "check_point": 1624517720}'
handler: 4 4 b'{"status": "successful", "idx": 2949, "check_point": 1624517720}'
handler: 5 5 b'{"status": "successful", "idx": 2949, "check_point": 1624517720}'
handler: 1 6 b'{"status": "successful", "idx": 2949, "check_point": 1624517720}'
handler: 3 7 b'{"status": "successful", "idx": 2949, "check_point": 1624517720}'
handler: 6 1 b'{"status": "successful", "idx": 2950, "check_point": 1624517725}'
handler: 4 2 b'{"status": "successful", "idx": 2950, "check_point": 1624517725}'
handler: 0 3 b'{"status": "successful", "idx": 2950, "check_point": 1624517725}'
handler: 3 4 b'{"status": "s

handler: 3 1 b'{"status": "successful", "idx": 2964, "check_point": 1624517795}'
handler: 0 2 b'{"status": "successful", "idx": 2964, "check_point": 1624517795}'
handler: 4 3 b'{"status": "successful", "idx": 2964, "check_point": 1624517795}'
handler: 2 4 b'{"status": "successful", "idx": 2964, "check_point": 1624517795}'
handler: 6 5 b'{"status": "successful", "idx": 2964, "check_point": 1624517795}'
handler: 5 6 b'{"status": "successful", "idx": 2964, "check_point": 1624517795}'
handler: 1 7 b'{"status": "successful", "idx": 2964, "check_point": 1624517795}'
handler: 4 1 b'{"status": "successful", "idx": 2965, "check_point": 1624517800}'
handler: 0 2 b'{"status": "successful", "idx": 2965, "check_point": 1624517800}'
handler: 6 3 b'{"status": "successful", "idx": 2965, "check_point": 1624517800}'
handler: 2 4 b'{"status": "successful", "idx": 2965, "check_point": 1624517800}'
handler: 3 5 b'{"status": "successful", "idx": 2965, "check_point": 1624517800}'
handler: 1 6 b'{"status": "s

handler: 0 1 b'{"status": "successful", "idx": 2979, "check_point": 1624517870}'
handler: 5 2 b'{"status": "successful", "idx": 2979, "check_point": 1624517870}'
handler: 4 3 b'{"status": "successful", "idx": 2979, "check_point": 1624517870}'
handler: 6 4 b'{"status": "successful", "idx": 2979, "check_point": 1624517870}'
handler: 3 5 b'{"status": "successful", "idx": 2979, "check_point": 1624517870}'
handler: 1 6 b'{"status": "successful", "idx": 2979, "check_point": 1624517870}'
handler: 2 7 b'{"status": "successful", "idx": 2979, "check_point": 1624517870}'
handler: 4 1 b'{"status": "successful", "idx": 2980, "check_point": 1624517875}'
handler: 0 2 b'{"status": "successful", "idx": 2980, "check_point": 1624517875}'
handler: 1 3 b'{"status": "successful", "idx": 2980, "check_point": 1624517875}'
handler: 6 4 b'{"status": "successful", "idx": 2980, "check_point": 1624517875}'
handler: 3 5 b'{"status": "successful", "idx": 2980, "check_point": 1624517875}'
handler: 5 6 b'{"status": "s

handler: 6 6 b'{"status": "successful", "idx": 2993, "check_point": 1624517940}'
handler: 1 7 b'{"status": "successful", "idx": 2993, "check_point": 1624517940}'
handler: 0 1 b'{"status": "successful", "idx": 2994, "check_point": 1624517945}'
handler: 4 2 b'{"status": "successful", "idx": 2994, "check_point": 1624517945}'
handler: 6 3 b'{"status": "successful", "idx": 2994, "check_point": 1624517945}'
handler: 5 4 b'{"status": "successful", "idx": 2994, "check_point": 1624517945}'
handler: 3 5 b'{"status": "successful", "idx": 2994, "check_point": 1624517945}'
handler: 2 6 b'{"status": "successful", "idx": 2994, "check_point": 1624517945}'
handler: 1 7 b'{"status": "successful", "idx": 2994, "check_point": 1624517945}'
handler: 5 1 b'{"status": "successful", "idx": 2995, "check_point": 1624517950}'
handler: 4 2 b'{"status": "successful", "idx": 2995, "check_point": 1624517950}'
handler: 3 3 b'{"status": "successful", "idx": 2995, "check_point": 1624517950}'
handler: 2 4 b'{"status": "s

In [5]:
rd.set('hq_assist_count', 10)

True